In [1]:
pip install numpy;

In [2]:
pip install pandas

In [3]:
pip install scikit-learn

In [4]:
pip install matplotlib

In [5]:
pip install seaborn

In [36]:
pip install tenserflow


ERROR: Could not find a version that satisfies the requirement tenserflow (from versions: none)
ERROR: No matching distribution found for tenserflow


In [7]:
pip install Flask

In [8]:
pip install kaggle


In [16]:
!mkdir ~/.kaggle

In [17]:
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [18]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import pandas as pd



In [19]:
IMAGE_SIZE=(224,224,3)

In [20]:
IMAGE_SIZE=(224,224,3)

In [38]:
train_data = "./data/train"
val_data = "./data/val"
test_data = "./data/test"

In [39]:
labels = ['Salmonella', 'New Castle Disease', 'Coccidiosis', 'Healthy']

In [42]:
import os
import cv2
# Correct relative paths
train_data = "archive/Chicken and Calf Diseases Dataset/data/train"
val_data = "archive/Chicken and Calf Diseases Dataset/data/val"
test_data = "archive/Chicken and Calf Diseases Dataset/data/test"

# Make sure you define this
labels = ['Coccidiosis', 'Healthy', 'New Castle Disease', 'Salmonella']

# Function to read image data
def read_data(folder):
    data, label, paths = [], [], []

    for l in labels:
        path = f"{folder}/{l}/"
        # Check if the directory exists before listing its contents
        if os.path.exists(path):
            folder_data = os.listdir(path)[:500]

            for image_path in folder_data:
                img = cv2.imread(os.path.join(path, image_path)) # Use os.path.join for creating paths
                data.append(img)
                label.append(l)
                paths.append(os.path.join(folder, l, image_path))
        else:
            print(f"Warning: Directory not found: {path}")


    return data, label, paths

# Call the function
all_data, all_labels, all_paths = read_data(train_data)

In [43]:
train_df = pd.DataFrame({
    'image': all_data,
    'path': all_paths,
    'label': all_labels
})

train_df


,image,path,label


In [44]:
# Reading test data
all_data, all_labels, all_paths = read_data(test_data)

# Creating test DataFrame
test_df = pd.DataFrame({
    'image': all_data,
    'path': all_paths,
    'label': all_labels
})

# Reading validation data
all_data, all_labels, all_paths = read_data(val_data)

# Creating validation DataFrame
val_df = pd.DataFrame({
    'image': all_data,
    'path': all_paths,
    'label': all_labels
})

In [49]:
import os

# List the contents of the training data directory to check for label subdirectories
train_data_path = "archive/Chicken and Calf Diseases Dataset/data/train"
if os.path.exists(train_data_path):
    print(f"Contents of '{train_data_path}':")
    print(os.listdir(train_data_path))
else:
    print(f"Directory not found: {train_data_path}")

Directory not found: archive/Chicken and Calf Diseases Dataset/data/train


In [50]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import tensorflow as tf

# Load pre-trained VGG16 model without top layer
vgg = VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

# Freeze the layers
for layer in vgg.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [51]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,80

 Total params: 15,772,996 (60.17 MB)

 Trainable params: 1,055,236 (4.03 MB)

 Non-trainable params: 14,717,760 (56.14 MB)

In [53]:
import os

# List the contents of the 'archive/Chicken and Calf Diseases Dataset' directory
dataset_base_path = "archive/Chicken and Calf Diseases Dataset"
if os.path.exists(dataset_base_path):
    print(f"Contents of '{dataset_base_path}':")
    print(os.listdir(dataset_base_path))
else:
    print(f"Directory not found: {dataset_base_path}")

Directory not found: archive/Chicken and Calf Diseases Dataset


In [58]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

def predictor(model, test_gen):
    classes = list(test_gen.class_indices.keys())
    class_count = len(classes)

    preds = model.predict(test_gen, verbose=1)
    errors = 0
    pred_indices = []
    test_count = len(preds)

    for i, p in enumerate(preds):
        pred_index = np.argmax(p)
        pred_indices.append(pred_index)
        true_index = test_gen.labels[i]
        if pred_index != true_index:
            errors += 1

    accuracy = (test_count - errors) * 100 / test_count
    ytrue = np.array(test_gen.labels, dtype='int')
    ypred = np.array(pred_indices, dtype='int')

    msg = f'There were {errors} errors in {test_count} tests for an accuracy of {accuracy:6.2f}'
    print(msg)

    cm = confusion_matrix(ytrue, ypred)

    # Plot the confusion matrix
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)
    plt.xticks(np.arange(class_count) + 0.5, classes, rotation=90)
    plt.yticks(np.arange(class_count) + 0.5, classes, rotation=0)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
    clr = classification_report(ytrue, ypred, target_names=classes, digits=4)
    print("Classification Report:\n-----------------------\n", clr)

In [1]:

from keras.layers import GlobalAveragePooling2D

In [2]:

!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [3]:

import kerastuner as kt

/tmp/ipython-input-3-1547854853.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt
